# EDA & Model Development 

In [124]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import random

## Initial Setup

In [125]:
#https://raw.githubusercontent.com/jchristo12/fantasy_football/master/data/full_data.csv
df = pd.read_csv('https://github.com/jchristo12/fantasy_football/blob/master/data/full_data.csv?raw=true')

In [126]:
df.columns

Index(['pk', 'gid', 'seas', 'wk', 'player', 'fname', 'lname', 'full_name',
       'team', 'pos1',
       ...
       'ou', 'sprv', 'ptsv', 'ptsh', 'udog', 'gen_cond', 'udog_binary',
       'gen_dv', 'def_team', 'f_pts'],
      dtype='object', length=171)

In [127]:
#remove rows that have NaN for the shifted variables
df_clean = df[~df.loc[:,'seas_pa':'seas_tdret'].isna().all(axis=1)]

##### We are only focused on offensive players right now. Therefore, we will specifiy the positions we want to retain in the data and remove everyone else

In [128]:
#store positions we are concerned about; will use these to filter out 
pos_of_interest = ['QB', 'RB', 'WR', 'TE']
#filter out positions we don't care about
df_clean2 = df_clean[df_clean['pos1'].isin(pos_of_interest)]

##### Convert each column to the appropriate data type (i.e. make the categorical data categories)

In [129]:
#set the column types
col_dtypes = {'category': ['seas', 'wk', 'pos1', 'team', 'udog', 'v', 'h', 'day', 'stad', 'wdir',
                          'surf', 'gen_cond', 'gen_dv', 'def_team']}
#flip the key and values around so they will work in the argument for 'astype()'
col_dtypes_alt = {old: new for new, old_all in col_dtypes.items() for old in old_all}

In [130]:
df_clean2 = df_clean2.astype(col_dtypes_alt)

##### Remove rookies

In [131]:
#store dataframe of non-rookies
df_clean2_vet = df_clean2.loc[df_clean2['exp']!=1, :]
#store dataframe of rookie data
df_clean2_rook = df_clean2.loc[df_clean2['exp']==1, :]

##### Segment out the WR data

Non-rookies data only

In [132]:
df_wr = df_clean2_vet.loc[df_clean2_vet['pos1']=='WR', :]

##### Only focus on week 10 data

In [133]:
df_wr10 = df_wr#.loc[df_wr['wk'] == 10, :]

In [134]:
df_wr10.shape

(28605, 171)

## EDA

Only focusing on the WR data right now. Will need to build this so that each action is extendable to the other positions

### Training/Test split

In [135]:
from sklearn.model_selection import train_test_split

In [136]:
#set the random seed for reproducability
random.seed(837)

In [137]:
#break out the data between training and test
train_wr, test_wr = train_test_split(df_wr10, train_size=0.75, test_size=0.25, shuffle=True)

In [138]:
#shape of the data
print(train_wr.shape, test_wr.shape)

(21453, 171) (7152, 171)


### Missing Data analysis

#### Basic analysis

In [139]:
#create a series of percent of missing data
missing_data_pct = train_wr.isna().sum() / train_wr.shape[0]

In [140]:
#list all columns with missing data that is greater than 25%
missing_data_pct[missing_data_pct > 0.25].sort_values(ascending=False)

Series([], dtype: float64)

In [142]:
#columns with missing values but less than or equal to 25%
impute_cols = missing_data_pct[(missing_data_pct <= 0.25) & (missing_data_pct > 0)].sort_values(ascending=False)
impute_cols

temp    0.166643
humd    0.039855
wdir    0.028900
dtype: float64

#### Take action on missing data

##### Drop columns with too many missing values

In [143]:
#drop columns with missing data greater than 25%
#store the column names
missing_cols_del = missing_data_pct[missing_data_pct > 0.25].sort_values(ascending=False).index
#drop the columns and store as new dataframe
train_wr_miss = train_wr.drop(missing_cols_del, axis=1, inplace=False)

In [144]:
#check the shape
train_wr_miss.shape

(21453, 171)

##### Impute the rest of the missing values

In [145]:
from sklearn.impute import SimpleImputer

In [146]:
#Build simple imputers for both numeric and categorical features
numeric_impute = SimpleImputer(missing_values=np.NaN, strategy='median')
cat_impute = SimpleImputer(missing_values=np.NaN, strategy='most_frequent')

In [147]:
#create a dataframe of all of the features to impute
missing_values_df = train_wr_miss.drop(train_wr_miss.columns.difference(impute_cols.index), axis=1, inplace=False)

In [148]:
missing_values_df.columns

Index(['temp', 'wdir', 'humd'], dtype='object')

In [149]:
#store the columns that need to use the numeric imputation and the categorical imputation
impute_numeric_col = missing_values_df.select_dtypes(include=np.number).columns
impute_cat_col = missing_values_df.select_dtypes(exclude=np.number).columns

This code below attempts to handle infinity values; This is not an issue right now but something to be aware of

In [150]:
#Boolean if the column has 'inf' values or not
#inf_cols = np.isinf(train_wr_miss.loc[:, impute_numeric_col]).any()

In [151]:
#list of columns that have 'inf' values
#inf_cols2 = list(train_wr_miss.loc[:, impute_numeric_col].columns.to_series()[inf_cols].values)

Continue with simple imputation

In [152]:
#impute numeric columns
imputed_numeric_df = pd.DataFrame(numeric_impute.fit_transform(train_wr_miss.loc[:, impute_numeric_col]), columns=impute_numeric_col)

In [153]:
#impute categorical columns
imputed_cat_df = pd.DataFrame(cat_impute.fit_transform(train_wr_miss.loc[:, impute_cat_col]), columns=impute_cat_col)